## TODO

- Add validation for metadata -> should be a separate heading (validate alignment across ALL datasets: should be identical)
- Update Dataset class so it doesn't have to take prefix or usecols

In [ ]:
%load_ext rich
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import numpy as np
import polars as pl
import pandera.polars as pa
from pandera.typing import DataFrame, Series
from typing import Any
import pyreadstat
from functools import partial

from pain.read import *
from fastcore.test import *

In [ ]:
from metadata import METADATA
data_dir = Path("../data/raw")

## Data Schema

Define expected data structure for each variable

In [ ]:
PN17 = partial(pa.Field, isin=(-99, 0, 1), coerce=True)
PN25 = partial(pa.Field, isin=(-88, -99, 0, 1), coerce=True)
PN34 = partial(pa.Field, isin=(-88, -99, 0, 1), coerce=True)
PN35 = partial(pa.Field, isin=(-88, -99, 0, 1), coerce=True)
PN36 = partial(pa.Field, isin=(-88, -99, 0, 1), coerce=True)

# Only applicable for G217_PQ and G217_SQ
PN9 = partial(pa.Field, isin=(-88, -99, 0, 1), coerce=True)
PN38 = partial(pa.Field, isin=(-88, -99, 0, 1), coerce=True)

## G214_PQ

In [ ]:
G214_PQ = Dataset("G214_PQ.sav", data_dir)
df, meta = G214_PQ.load_data()

In [ ]:
df = (
    df
    .select(
        pl.col("G214_PQ_PN17").replace({9: -99}),
        pl.col("G214_PQ_PN25").replace({8: -88, 9: -99}),
        pl.col("G214_PQ_PN34").replace({8: -88, 9: -99}),
        pl.col("G214_PQ_PN35").replace({8: -88, 9: -99}),
        pl.col("G214_PQ_PN36").replace({8: -88, 9: -99})
    )
).collect()

In [ ]:
class G214PQSchema(pa.DataFrameModel):
    G214_PQ_PN17: Series[int] = PN17()
    G214_PQ_PN25: Series[int] = PN25()
    G214_PQ_PN34: Series[int] = PN34()
    G214_PQ_PN35: Series[int] = PN35()
    G214_PQ_PN36: Series[int] = PN36()

    @pa.dataframe_check
    def check_for_na(cls, data: pa.PolarsData) -> pl.LazyFrame:
        """Verify when PN17 is 0, all subsequent values are -88 (ie. if no back pain, other variables should be N/A)."""
        s = pl.col("G214_PQ_PN25", "G214_PQ_PN34", "G214_PQ_PN35", "G214_PQ_PN36")
        return data.lazyframe.filter(pl.col("G214_PQ_PN17") == 0).select(s == -88)
    
    @pa.dataframe_check
    def check_for_na2(cls, data: pa.PolarsData) -> pl.LazyFrame:
        """
        Verify the reverse of the above, where if any subsequent variables has a value of -88, PN17 should be 0.
        Except PN35, where there are valid values of -88 when PN17 is 0
        """
        f = ((pl.col("G214_PQ_PN25") == -88) |
             (pl.col("G214_PQ_PN34") == -88) |
             (pl.col("G214_PQ_PN36") == -88))
        return data.lazyframe.filter(f).select(pl.col("G214_PQ_PN17") == 0)

try:
    G214PQSchema.validate(df, lazy=True)
except pa.errors.SchemaErrors as err:
    print(err)

## Explore Metadata Validation

One sensible approach would be to take a collection of Metadata and transform it so rather than having each variable defined independently, in isolation, that data is converted to be in the same format as the current metadata structure of the dataset (ie. Parameters as parents, variables as children)

In [ ]:
new_meta = convert_metadata_list_to_dict(METADATA, p="G214_PQ_")
m = merge_dictionaries([new_meta, meta])

In [ ]:
from pydantic import BaseModel, ValidationError, field_validator

# General validation class for metadata in nested dictionary format
class MetaDict(BaseModel):
    label: dict[str, str|None] # TODO: replace None with empty string? Valid error - should have label?
    field_values: dict[str, dict[int, str]]
    field_type: dict[str, str] # TODO: Add validator to ensure values are one of ["Numeric", "String", "Date"]
    field_width: dict[str, int]
    decimals: dict[str, int]
    variable_type: dict[str, str]

    # TODO: validator fail because they're trying to read individual elements, not the larger dict
    # Can I create BaseModels for each parameter? Hard without knowing and specifying the individual variables

    # @field_validator('variable_type', mode='before')
    # @classmethod
    # def lowercase(cls, value: str) -> str:
    #     return value.lower()

    # @field_validator('field_type', mode='before')
    # @classmethod
    # def check_field_type(cls, value: str) -> str:
    #     if value in ["Numeric", "String", "Date"]:
    #         return value
    #     else:
    #         raise ValueError(f"{value} not one of 'Numeric', 'String' or 'Date'.")

    # TODO: check that the keys of each dict are identical

In [ ]:
#| export
def munge_keys(m: dict[str, dict[str, Any]]) -> dict[str, dict[str, Any]]:
    """Munge outer keys of dictionary for use with Pydantic."""
    return {k.lower().replace(" ", "_"): v for k, v in m.items()}

In [ ]:
MetaDict(**munge_keys(new_meta))


MetaDict(
    label={
        'G214_PQ_PN17': 'Ever had back pain',
        'G214_PQ_PN25': 'Sought professional advice/treatment',
        'G214_PQ_PN34': 'Took medication to relieve pain',
        'G214_PQ_PN35': 'Missed work due to pain',
        'G214_PQ_PN36': 'Pain interfered with normal activities',
        'G214_PQ_PN9': 'Ever had neck/shoulder pain',
        'G214_PQ_PN38': 'Ever had low back pain'
    },
    field_values={
        'G214_PQ_PN17': {-99: 'Missing', 0: 'No', 1: 'Yes'},
        'G214_PQ_PN25': {-88: 'N/A', -99: 'Missing', 0: 'No', 1: 'Yes'},
        'G214_PQ_PN34': {-88: 'N/A', -99: 'Missing', 0: 'No', 1: 'Yes'},
        'G214_PQ_PN35': {-88: 'N/A', -99: 'Missing', 0: 'No', 1: 'Yes'},
        'G214_PQ_PN36': {-88: 'N/A', -99: 'Missing', 0: 'No', 1: 'Yes'},
        'G214_PQ_PN9': {-99: 'Missing', 0: 'No', 1: 'Yes'},
        'G214_PQ_PN38': {-99: 'Missing', 0: 'No', 1: 'Yes'}
    },
    field_type={
        'G214_PQ_PN17': 'Numeric',
        'G214_PQ_PN25': 'Numer

In [ ]:
# Munge variable names to suit Pydantic
# TODO: convert this into an actual reusable function
m2 = munge_keys(m)
try:
    MetaDict(**m2)
except ValidationError as err:
    print(err)

In [ ]:
class Label(BaseModel):
    G214_PQ_PN17: str = 'Ever had back pain'
    G214_PQ_PN25: str = 'Sought professional advice/treatment'
    G214_PQ_PN34: str = 'Took medication to relieve pain'
    G214_PQ_PN35: str = 'Missed work due to pain'
    G214_PQ_PN36: str = 'Pain interfered with normal activities'

In [ ]:
class Meta(BaseModel):
    label: Label

In [ ]:
Meta(**munge_keys(new_meta))


Meta(
    label=Label(
        G214_PQ_PN17='Ever had back pain',
        G214_PQ_PN25='Sought professional advice/treatment',
        G214_PQ_PN34='Took medication to relieve pain',
        G214_PQ_PN35='Missed work due to pain',
        G214_PQ_PN36='Pain interfered with normal activities'
    )
)


## G214_SQ

In [ ]:
G214_SQ = Dataset("G214_SQ.sav", data_dir)
df, _ = G214_SQ.load_data()

In [ ]:
df = (
    df
    .with_columns(
        pl.col("G214_SQ_PN17").replace({9: -99}),
        pl.col("G214_SQ_PN25").replace({8: -88, 9: -99}),
        pl.col("G214_SQ_PN34").replace({8: -88, 9: -99}),
        pl.col("G214_SQ_PN35").replace({8: -88, 9: -99}),
        pl.col("G214_SQ_PN36").replace({8: -88, 9: -99}),
    )
)

In [ ]:
class DataSchema_G214_SQ(pa.DataFrameModel):
    G214_SQ_PN17: Series[int] = PN17(nullable=True)
    G214_SQ_PN25: Series[int] = PN25(nullable=True)
    G214_SQ_PN34: Series[int] = PN34(nullable=True)
    G214_SQ_PN35: Series[int] = PN35(nullable=True)
    G214_SQ_PN36: Series[int] = PN36(nullable=True)

    @pa.dataframe_check
    def check_for_na(cls, data: pa.PolarsData) -> pl.LazyFrame:
        """Verify when PN17 is 0, all subsequent values are -88 (ie. if no back pain, other variables should be N/A)."""
        s = pl.col("G214_SQ_PN25", "G214_SQ_PN34", "G214_SQ_PN35", "G214_SQ_PN36")
        return data.lazyframe.filter(pl.col("G214_SQ_PN17") == 0).select(s == -88)
    
    @pa.dataframe_check
    def check_for_na2(cls, data: pa.PolarsData) -> pl.LazyFrame:
        """
        Verify the reverse of the above, where if any subsequent variables has a value of -88, PN17 should be 0.
        Except PN35, where there are valid values of -88 when PN17 is 0
        """
        f = ((pl.col("G214_SQ_PN25") == -88) |
             (pl.col("G214_SQ_PN34") == -88) |
             (pl.col("G214_SQ_PN36") == -88))
        return data.lazyframe.filter(f).select(pl.col("G214_SQ_PN17") == 0)

try:
    DataSchema_G214_SQ.validate(df.collect(), lazy=True)
except pa.errors.SchemaErrors as err:
    print(err)

## G217_PQ

In [ ]:
G217_PQ = Dataset("G217_PQ.sav", data_dir)
df, _ = G217_PQ.load_data()

In [ ]:
df = (
    df
    .select(
        pl.col("G217_PQ_PN17").replace({9: -99}),
        pl.col("G217_PQ_PN9").replace({7: -99, 9: -99}),
        pl.col("G217_PQ_PN38").replace({7: -99, 9: -99}),
        pl.col("G217_PQ_PN25").replace({7: -99, 9: -99}),
        pl.col("G217_PQ_PN34").replace({7: -99, 9: -99}),
        pl.col("G217_PQ_PN35").replace({7: -99, 9: -99}),
        pl.col("G217_PQ_PN36").replace({7: -99, 9: -99})
    )
)

In [ ]:
class G217PQSchema(pa.DataFrameModel):
    G217_PQ_PN17: Series[int] = PN17()
    G217_PQ_PN9: Series[int] = PN9()
    G217_PQ_PN38: Series[int] = PN38()
    G217_PQ_PN25: Series[int] = PN25()
    G217_PQ_PN34: Series[int] = PN34()
    G217_PQ_PN35: Series[int] = PN35()
    G217_PQ_PN36: Series[int] = PN36()

try:
    df = G217PQSchema.validate(df, lazy=True)
except pa.errors.SchemaErrors as err:
    print(err)

## G217_SQ

In [ ]:
G217_SQ = Dataset("G217_SQ.sav", data_dir)
df, _ = G217_SQ.load_data()

In [ ]:
df = (
    df
    .select(
        pl.col("G217_SQ_PN17").replace({9: -99}),
        pl.col("G217_SQ_PN9").replace({9: -99}),
        pl.col("G217_SQ_PN38").replace({9: -99}),
        pl.col("G217_SQ_PN25").replace({9: -99}),
        pl.col("G217_SQ_PN34").replace({9: -99}),
        pl.col("G217_SQ_PN35").replace({9: -99}),
        pl.col("G217_SQ_PN36").replace({9: -99})
    )
)

In [ ]:
class G217SQDataSchema(pa.DataFrameModel):
    G217_SQ_PN17: Series[int] = PN17()
    G217_SQ_PN9: Series[int] = PN9()
    G217_SQ_PN38: Series[int] = PN38()
    G217_SQ_PN25: Series[int] = PN25()
    G217_SQ_PN34: Series[int] = PN34()
    G217_SQ_PN35: Series[int] = PN35()
    G217_SQ_PN36: Series[int] = PN36()

try:
    df = G217SQDataSchema.validate(df, lazy=True)
except pa.errors.SchemaErrors as err:
    print(err)